# Code Setup

In [154]:
import numpy as np

from utils.import_puzzle_input import load_input
from utils.import_puzzle_input import split_puzzle_input
from utils.import_puzzle_input import matrix_puzzle_input

from functools import reduce

# Load the autoreload extension
%load_ext autoreload

# Set autoreload to automatically reload modules before executing code
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [155]:
puzzle_input = load_input(2023, 5)

loaded puzzle input


In [156]:
puzzle_input_split = split_puzzle_input(puzzle_input)

In [157]:
puzzle_input_split

['seeds: 2041142901 113138307 302673608 467797997 1787644422 208119536 143576771 99841043 4088720102 111819874 946418697 13450451 3459931852 262303791 2913410855 533641609 2178733435 26814354 1058342395 175406592',
 '',
 'seed-to-soil map:',
 '1270068015 1235603193 242614277',
 '13415696 1478217470 21049126',
 '825250550 1160341941 75261252',
 '3786189027 1971702238 242038712',
 '3191605340 3433644052 172752250',
 '2389904665 3088515862 345128190',
 '0 1499266596 13415696',
 '1197451933 0 72616082',
 '2139929050 1721726623 249975615',
 '900511802 222541761 147014452',
 '1047526254 72616082 149925679',
 '34464822 980591812 179750129',
 '2735032855 2631943377 456572485',
 '3364357590 3606396302 421831437',
 '214214951 369556213 611035599',
 '1721726623 2213740950 418202427',
 '',
 'soil-to-fertilizer map:',
 '226793587 358613369 356867344',
 '0 1838890301 226793587',
 '2741010192 0 358613369',
 '2257843811 715480713 173982825',
 '3099623561 1264222741 3082010',
 '1810570233 2912833547 32

# Problem Setup

--- Day 5: If You Give A Seed A Fertilizer ---  
You take the boat and find the gardener right where you were told he would be: managing a giant "garden" that looks more to you like a farm.  
  
"A water source? Island Island is the water source!" You point out that Snow Island isn't receiving any water.  
  
"Oh, we had to stop the water because we ran out of sand to filter it with! Can't make snow with dirty water. Don't worry, I'm sure we'll get more sand soon; we only turned off the water a few days... weeks... oh no." His face sinks into a look of horrified realization.  
  
"I've been so busy making sure everyone here has food that I completely forgot to check why we stopped getting more sand! There's a ferry leaving soon that is headed over in that direction - it's much faster than your boat. Could you please go check it out?"  
  
You barely have time to agree to this request when he brings up another. "While you wait for the ferry, maybe you can help us with our food production problem. The latest Island Island Almanac just arrived and we're having trouble making sense of it."  
  
The almanac (your puzzle input) lists all of the seeds that need to be planted. It also lists what type of soil to use with each kind of seed, what type of fertilizer to use with each kind of soil, what type of water to use with each kind of fertilizer, and so on. Every type of seed, soil, fertilizer and so on is identified with a number, but numbers are reused by each category - that is, soil 123 and fertilizer 123 aren't necessarily related to each other.  
  
For example:  
  
seeds: 79 14 55 13  
  
seed-to-soil map:  
50 98 2  
52 50 48  
  
soil-to-fertilizer map:  
0 15 37  
37 52 2  
39 0 15  
  
fertilizer-to-water map:  
49 53 8  
0 11 42  
42 0 7  
57 7 4  
  
water-to-light map:  
88 18 7  
18 25 70  
  
light-to-temperature map:  
45 77 23  
81 45 19  
68 64 13  
  
temperature-to-humidity map:  
0 69 1  
1 0 69  
  
humidity-to-location map:  
60 56 37  
56 93 4  
The almanac starts by listing which seeds need to be planted: seeds 79, 14, 55, and 13.  
  
The rest of the almanac contains a list of maps which describe how to convert numbers from a source category into numbers in a destination category. That is, the section that starts with seed-to-soil map: describes how to convert a seed number (the source) to a soil number (the destination). This lets the gardener and his team know which soil to use with which seeds, which water to use with which fertilizer, and so on.  
  
Rather than list every source number and its corresponding destination number one by one, the maps describe entire ranges of numbers that can be converted. Each line within a map contains three numbers: the destination range start, the source range start, and the range length.  
  
Consider again the example seed-to-soil map:  
  
50 98 2  
52 50 48  
The first line has a destination range start of 50, a source range start of 98, and a range length of 2. This line means that the source range starts at 98 and contains two values: 98 and 99. The destination range is the same length, but it starts at 50, so its two values are 50 and 51. With this information, you know that seed number 98 corresponds to soil number 50 and that seed number 99 corresponds to soil number 51.  
  
The second line means that the source range starts at 50 and contains 48 values: 50, 51, ..., 96, 97. This corresponds to a destination range starting at 52 and also containing 48 values: 52, 53, ..., 98, 99. So, seed number 53 corresponds to soil number 55.  
  
Any source numbers that aren't mapped correspond to the same destination number. So, seed number 10 corresponds to soil number 10.  
  
So, the entire list of seed numbers and their corresponding soil numbers looks like this:  
  
seed  soil  
0     0  
1     1  
...   ...  
48    48  
49    49  
50    52  
51    53  
...   ...  
96    98  
97    99  
98    50  
99    51  
With this map, you can look up the soil number required for each initial seed number:  
  
Seed number 79 corresponds to soil number 81.  
Seed number 14 corresponds to soil number 14.  
Seed number 55 corresponds to soil number 57.  
Seed number 13 corresponds to soil number 13.  
The gardener and his team want to get started as soon as possible, so they'd like to know the closest location that needs a seed. Using these maps, find the lowest location number that corresponds to any of the initial seeds. To do this, you'll need to convert each seed number through other categories until you can find its corresponding location number. In this example, the corresponding types are:  
  
Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82.  
Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43.  
Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86.  
Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35.  
So, the lowest location number in this example is 35.  
  
What is the lowest location number that corresponds to any of the initial seed numbers?  

# General thoughts

We might want to assume that there is a unique trace-back of categories such that there are not multiple categories mapping to, for example, location.

# Solutions

## Solution 1:

### Explanation of method

1. RETRIEVING THE TYPE MAPPINGS:  
1.1. Find lines containing the word `to`    
1.2. Add each *source-to-destination* *tag-mapping* as a key to the `tag_mapping_dict` dictionary.   
  
2. GETTING THE TRACE:    
2.0. Write a function `get_source_tag` that returns the *source-tag* to a particular *destination-tag*  
2.1. Find the *location*-tag (presumably unique)  
2.2. Use `get_source_tag` to retrieve the *source-tag* corresponding to having *location* as its *destination-tag*. This source tag will be the destination tag to some (presumably unique) new *source-tag*  
2.3. Repeat 2.2 until we reach *seed*. Retrieve the *tag-mappings*-chain, as the reverse order of the inverse-chain.  
  
3. APPLYING THE MAPPINGS:  
3.0.  
3.1. 

### Implementation

### Step 1:

In [158]:
def get_mapping_from_tuple(mapping_tuple: tuple):
    start_destination = mapping_tuple[0]
    start_source = mapping_tuple[1]
    length_custom_mapping = mapping_tuple[2]

    range_source = range(start_source, start_source + length_custom_mapping)
    range_destination = range(start_destination, start_destination + length_custom_mapping)
    source_destination_mapping_dict = {source: destination for source, destination in zip(range_source, range_destination)}
    
    source_destination_mapping_dict.values()
    # Add non-custom mappings that are one to one (take 0 to 99 range and setminus range_destination)


### Step 2:

### Checking runtime

#### Benchmarking

#### Runtime profiling

## Solution 2:

# Experimenting

## Experimenting with wrangling the data

In [159]:
len(list(range(50, 50+48)))

48

In [160]:
# test_tmp.keys()

In [161]:
def test_get_source_destination_mapping_01(mapping_tuple: tuple):
    start_destination = mapping_tuple[0]
    start_source = mapping_tuple[1]
    length_custom_mapping = mapping_tuple[2]

    range_source = range(start_source, start_source + length_custom_mapping)
    range_destination = range(start_destination, start_destination + length_custom_mapping)
    source_destination_mapping_dict = {source: destination for source, destination in zip(range_source, range_destination)}
    
    return source_destination_mapping_dict

In [162]:
test_mapping_tuple_01 = (50, 98, 2)

In [163]:
test_get_source_destination_mapping_01(test_mapping_tuple_01)

{98: 50, 99: 51}

In [164]:
test_mapping_tuple_02 = (52, 50, 48)

In [165]:
test_get_source_destination_mapping_01(test_mapping_tuple_02)

{50: 52,
 51: 53,
 52: 54,
 53: 55,
 54: 56,
 55: 57,
 56: 58,
 57: 59,
 58: 60,
 59: 61,
 60: 62,
 61: 63,
 62: 64,
 63: 65,
 64: 66,
 65: 67,
 66: 68,
 67: 69,
 68: 70,
 69: 71,
 70: 72,
 71: 73,
 72: 74,
 73: 75,
 74: 76,
 75: 77,
 76: 78,
 77: 79,
 78: 80,
 79: 81,
 80: 82,
 81: 83,
 82: 84,
 83: 85,
 84: 86,
 85: 87,
 86: 88,
 87: 89,
 88: 90,
 89: 91,
 90: 92,
 91: 93,
 92: 94,
 93: 95,
 94: 96,
 95: 97,
 96: 98,
 97: 99}

In [166]:
def test_get_source_destination_mapping_02(mapping_tuple: tuple):
    start_destination = mapping_tuple[0]
    start_source = mapping_tuple[1]
    length_custom_mapping = mapping_tuple[2]

    range_source = range(start_source, start_source + length_custom_mapping)
    range_destination = range(start_destination, start_destination + length_custom_mapping)
    source_destination_mapping_dict = {i: i for i in range(100)}

    source_destination_mapping_dict.update({source: destination for source, destination in zip(range_source, range_destination)})
    
    return source_destination_mapping_dict

In [167]:
test_get_source_destination_mapping_02(test_mapping_tuple_01)

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 8,
 9: 9,
 10: 10,
 11: 11,
 12: 12,
 13: 13,
 14: 14,
 15: 15,
 16: 16,
 17: 17,
 18: 18,
 19: 19,
 20: 20,
 21: 21,
 22: 22,
 23: 23,
 24: 24,
 25: 25,
 26: 26,
 27: 27,
 28: 28,
 29: 29,
 30: 30,
 31: 31,
 32: 32,
 33: 33,
 34: 34,
 35: 35,
 36: 36,
 37: 37,
 38: 38,
 39: 39,
 40: 40,
 41: 41,
 42: 42,
 43: 43,
 44: 44,
 45: 45,
 46: 46,
 47: 47,
 48: 48,
 49: 49,
 50: 50,
 51: 51,
 52: 52,
 53: 53,
 54: 54,
 55: 55,
 56: 56,
 57: 57,
 58: 58,
 59: 59,
 60: 60,
 61: 61,
 62: 62,
 63: 63,
 64: 64,
 65: 65,
 66: 66,
 67: 67,
 68: 68,
 69: 69,
 70: 70,
 71: 71,
 72: 72,
 73: 73,
 74: 74,
 75: 75,
 76: 76,
 77: 77,
 78: 78,
 79: 79,
 80: 80,
 81: 81,
 82: 82,
 83: 83,
 84: 84,
 85: 85,
 86: 86,
 87: 87,
 88: 88,
 89: 89,
 90: 90,
 91: 91,
 92: 92,
 93: 93,
 94: 94,
 95: 95,
 96: 96,
 97: 97,
 98: 50,
 99: 51}

We note that the above functions only deal with the first mapping tuple line. We will need an update function

In [168]:
def test_update_source_destination_mapping_01(mapping_tuple: tuple, previous_dict: dict):
    start_destination = mapping_tuple[0]
    start_source = mapping_tuple[1]
    length_custom_mapping = mapping_tuple[2]

    range_source = range(start_source, start_source + length_custom_mapping)
    range_destination = range(start_destination, start_destination + length_custom_mapping)
    
    new_dict = previous_dict.copy() # !!! IS THIS NECCESSARY?
    
    new_dict.update({source: destination for source, destination in zip(range_source, range_destination)})
    
    return new_dict

This ends up looking a lot like the previous functions. Let's generalise:

In [169]:
def test_create_source_destination_mapping_01(mapping_tuple: tuple, previous_dict: dict = None):
    start_destination = mapping_tuple[0]
    start_source = mapping_tuple[1]
    length_custom_mapping = mapping_tuple[2]

    range_source = range(start_source, start_source + length_custom_mapping)
    range_destination = range(start_destination, start_destination + length_custom_mapping)
    
    new_dict = previous_dict.copy() if previous_dict else {i: i for i in range(100)}
    
    new_dict.update({source: destination for source, destination in zip(range_source, range_destination)})
    
    return new_dict

In [170]:
test_mapping_tuple_result_01 = test_create_source_destination_mapping_01(test_mapping_tuple_01)
test_mapping_tuple_result_01

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 8,
 9: 9,
 10: 10,
 11: 11,
 12: 12,
 13: 13,
 14: 14,
 15: 15,
 16: 16,
 17: 17,
 18: 18,
 19: 19,
 20: 20,
 21: 21,
 22: 22,
 23: 23,
 24: 24,
 25: 25,
 26: 26,
 27: 27,
 28: 28,
 29: 29,
 30: 30,
 31: 31,
 32: 32,
 33: 33,
 34: 34,
 35: 35,
 36: 36,
 37: 37,
 38: 38,
 39: 39,
 40: 40,
 41: 41,
 42: 42,
 43: 43,
 44: 44,
 45: 45,
 46: 46,
 47: 47,
 48: 48,
 49: 49,
 50: 50,
 51: 51,
 52: 52,
 53: 53,
 54: 54,
 55: 55,
 56: 56,
 57: 57,
 58: 58,
 59: 59,
 60: 60,
 61: 61,
 62: 62,
 63: 63,
 64: 64,
 65: 65,
 66: 66,
 67: 67,
 68: 68,
 69: 69,
 70: 70,
 71: 71,
 72: 72,
 73: 73,
 74: 74,
 75: 75,
 76: 76,
 77: 77,
 78: 78,
 79: 79,
 80: 80,
 81: 81,
 82: 82,
 83: 83,
 84: 84,
 85: 85,
 86: 86,
 87: 87,
 88: 88,
 89: 89,
 90: 90,
 91: 91,
 92: 92,
 93: 93,
 94: 94,
 95: 95,
 96: 96,
 97: 97,
 98: 50,
 99: 51}

In [171]:
test_mapping_tuple_result_02 = test_create_source_destination_mapping_01(test_mapping_tuple_02, test_mapping_tuple_result_01)
test_mapping_tuple_result_02

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 8,
 9: 9,
 10: 10,
 11: 11,
 12: 12,
 13: 13,
 14: 14,
 15: 15,
 16: 16,
 17: 17,
 18: 18,
 19: 19,
 20: 20,
 21: 21,
 22: 22,
 23: 23,
 24: 24,
 25: 25,
 26: 26,
 27: 27,
 28: 28,
 29: 29,
 30: 30,
 31: 31,
 32: 32,
 33: 33,
 34: 34,
 35: 35,
 36: 36,
 37: 37,
 38: 38,
 39: 39,
 40: 40,
 41: 41,
 42: 42,
 43: 43,
 44: 44,
 45: 45,
 46: 46,
 47: 47,
 48: 48,
 49: 49,
 50: 52,
 51: 53,
 52: 54,
 53: 55,
 54: 56,
 55: 57,
 56: 58,
 57: 59,
 58: 60,
 59: 61,
 60: 62,
 61: 63,
 62: 64,
 63: 65,
 64: 66,
 65: 67,
 66: 68,
 67: 69,
 68: 70,
 69: 71,
 70: 72,
 71: 73,
 72: 74,
 73: 75,
 74: 76,
 75: 77,
 76: 78,
 77: 79,
 78: 80,
 79: 81,
 80: 82,
 81: 83,
 82: 84,
 83: 85,
 84: 86,
 85: 87,
 86: 88,
 87: 89,
 88: 90,
 89: 91,
 90: 92,
 91: 93,
 92: 94,
 93: 95,
 94: 96,
 95: 97,
 96: 98,
 97: 99,
 98: 50,
 99: 51}

## Post finding out the problem is not conditioned to be of numbers between 0 and 99

Ok, with no concrete bound and very large numbers and ranges, we cannot store the mappings explicitly

In [172]:
15 in range(10, 20)

True

In [173]:
x = 9
ranges = [range(10, 15), range(100, 200), range(5, 10)]
if any(x in r for r in ranges):
    print("x is in!")
else:
    print("x is not in!")

x is in!


In [174]:
tmp_bool = [x in r for r in ranges]
tmp_bool

[False, False, True]

In [175]:
tmp_range = range(5, 10)

In [176]:
tmp_range.start

5

In [177]:
tmp_range.stop

10

In [178]:
tmp_range.step

1

In [179]:
np.where(tmp_bool)[0][0]

np.int64(2)

In [180]:
tmp_correct_range = ranges[np.where(tmp_bool)[0][0]]
tmp_correct_range

range(5, 10)

In [181]:
source_number = 99
source_range_starts = [98, 50]
source_range_lengths = [2, 48]
source_ranges = [range(source_range_starts[0], source_range_starts[0] + source_range_lengths[0]), range(source_range_starts[1], source_range_starts[1] + source_range_lengths[1])]

In [182]:
source_ranges

[range(98, 100), range(50, 98)]

In [183]:
def create_list_of_ranges(range_start_list: list[int], range_length_list: list[int]):
    assert len(range_start_list) == len(range_length_list)
    ranges = [range(range_start_list[i], range_start_list[i] + range_length_list[i]) for i in range(len(range_start_list))]
    return ranges

In [184]:
def get_destination_source_ranges(source_range_start_list: list[int], destination_range_start_list: list[int], range_length_list: list[int]):
    source_ranges = create_list_of_ranges(source_range_start_list, range_length_list)
    destination_ranges = create_list_of_ranges(destination_range_start_list, range_length_list)
    return destination_ranges, source_ranges

In [185]:
source_range_starts = [98, 50]
destination_range_starts = [50, 52]
range_lengths = [2, 48]
print(create_list_of_ranges(source_range_starts, range_lengths))
print(create_list_of_ranges(destination_range_starts, range_lengths))
print(get_destination_source_ranges(source_range_starts, destination_range_starts, range_lengths))

[range(98, 100), range(50, 98)]
[range(50, 52), range(52, 100)]
([range(50, 52), range(52, 100)], [range(98, 100), range(50, 98)])


In [186]:
if any(source_number in r for r in source_ranges):
    print(f"source_number is in!")
else:
    print("source_number is not in!")

source_number is in!


In [187]:
[i for i, r in enumerate(source_ranges) if source_number in r]

[0]

In [188]:
(i for i, r in enumerate(source_ranges) if source_number in r)

<generator object <genexpr> at 0x000002404DFAC2B0>

In [189]:
source_diff_from_start = x - tmp_correct_range.start 
source_diff_from_start

4

In [190]:
source_number = 55
source_range_starts = [98, 50]
destination_range_starts = [50, 52]
range_lengths = [2, 48]
# print(create_list_of_ranges(source_range_starts, range_lengths))
# print(create_list_of_ranges(destination_range_starts, range_lengths))
# print(get_destination_source_ranges(source_range_starts, destination_range_starts, range_lengths))
destination_ranges, source_ranges = get_destination_source_ranges(source_range_starts, destination_range_starts, range_lengths)
source_ranges_index = next((i for i, r in reversed(list(enumerate(source_ranges))) if source_number in r), -1)
print(f"source_number {source_number} is in the source_range with index {source_ranges_index}, which is {source_ranges[source_ranges_index]}")
print(f"This means that the difference between the start of the range that contains the source number and the source number itself is {source_number - source_ranges[source_ranges_index].start}")
print(f"This means that the source number {source_number} will map to the destination number {destination_range_starts[source_ranges_index] + (source_number - source_ranges[source_ranges_index].start)}")

source_number 55 is in the source_range with index 1, which is range(50, 98)
This means that the difference between the start of the range that contains the source number and the source number itself is 5
This means that the source number 55 will map to the destination number 57


In [191]:
def get_destination_number(source_number: int, source_ranges: list[int], destination_ranges: list[int]) -> int:
    # If there are overlaps in source ranges, take the last one.
    source_ranges_index = next((i for i, r in reversed(list(enumerate(source_ranges))) if source_number in r), -1)

    if source_ranges_index != -1:
        # source_number is in a range
        correct_source_range = source_ranges[source_ranges_index]
        correct_source_range_start = correct_source_range.start
        source_number_diff_correct_source_range_start = source_number - correct_source_range_start
        destination_number = destination_ranges[source_ranges_index].start + source_number_diff_correct_source_range_start
    else:
        # source_number is not in a range, and should be mapped to the identity.
        destination_number = source_number
        
    return destination_number

## Experimenting with 'retrival' of data

### Retriving the seeds

In [192]:
puzzle_input_split

['seeds: 2041142901 113138307 302673608 467797997 1787644422 208119536 143576771 99841043 4088720102 111819874 946418697 13450451 3459931852 262303791 2913410855 533641609 2178733435 26814354 1058342395 175406592',
 '',
 'seed-to-soil map:',
 '1270068015 1235603193 242614277',
 '13415696 1478217470 21049126',
 '825250550 1160341941 75261252',
 '3786189027 1971702238 242038712',
 '3191605340 3433644052 172752250',
 '2389904665 3088515862 345128190',
 '0 1499266596 13415696',
 '1197451933 0 72616082',
 '2139929050 1721726623 249975615',
 '900511802 222541761 147014452',
 '1047526254 72616082 149925679',
 '34464822 980591812 179750129',
 '2735032855 2631943377 456572485',
 '3364357590 3606396302 421831437',
 '214214951 369556213 611035599',
 '1721726623 2213740950 418202427',
 '',
 'soil-to-fertilizer map:',
 '226793587 358613369 356867344',
 '0 1838890301 226793587',
 '2741010192 0 358613369',
 '2257843811 715480713 173982825',
 '3099623561 1264222741 3082010',
 '1810570233 2912833547 32

In [193]:
puzzle_input_split[0].split("seeds: ")

['',
 '2041142901 113138307 302673608 467797997 1787644422 208119536 143576771 99841043 4088720102 111819874 946418697 13450451 3459931852 262303791 2913410855 533641609 2178733435 26814354 1058342395 175406592']

In [194]:
test_seeds = list(map(int, puzzle_input_split[0].split("seeds: ")[1].split(" ")))
test_seeds

[2041142901,
 113138307,
 302673608,
 467797997,
 1787644422,
 208119536,
 143576771,
 99841043,
 4088720102,
 111819874,
 946418697,
 13450451,
 3459931852,
 262303791,
 2913410855,
 533641609,
 2178733435,
 26814354,
 1058342395,
 175406592]

In [195]:
def get_seeds(puzzle_input_split: list[str]) -> list[int]:
    seeds = list(map(int, puzzle_input_split[0].split("seeds: ")[1].split(" ")))
    return seeds

### Retrieve the mappings

In [196]:
puzzle_input_split

['seeds: 2041142901 113138307 302673608 467797997 1787644422 208119536 143576771 99841043 4088720102 111819874 946418697 13450451 3459931852 262303791 2913410855 533641609 2178733435 26814354 1058342395 175406592',
 '',
 'seed-to-soil map:',
 '1270068015 1235603193 242614277',
 '13415696 1478217470 21049126',
 '825250550 1160341941 75261252',
 '3786189027 1971702238 242038712',
 '3191605340 3433644052 172752250',
 '2389904665 3088515862 345128190',
 '0 1499266596 13415696',
 '1197451933 0 72616082',
 '2139929050 1721726623 249975615',
 '900511802 222541761 147014452',
 '1047526254 72616082 149925679',
 '34464822 980591812 179750129',
 '2735032855 2631943377 456572485',
 '3364357590 3606396302 421831437',
 '214214951 369556213 611035599',
 '1721726623 2213740950 418202427',
 '',
 'soil-to-fertilizer map:',
 '226793587 358613369 356867344',
 '0 1838890301 226793587',
 '2741010192 0 358613369',
 '2257843811 715480713 173982825',
 '3099623561 1264222741 3082010',
 '1810570233 2912833547 32

In [197]:
list(map(lambda x: "map" in x, puzzle_input_split))

[False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 Fals

In [198]:
tmp_list = ["a", "b", "c"]

In [199]:
for i, p in enumerate(tmp_list):
    print(f"i: {i}, p: {p}")

i: 0, p: a
i: 1, p: b
i: 2, p: c


In [200]:
test_map_index = [i for i, x in enumerate(puzzle_input_split) if "map" in x]
test_map_index

[2, 20, 40, 82, 100, 142, 182]

In [201]:
list(range(test_map_index[0] + 1, test_map_index[1] - 1))

[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [202]:
print(f"puzzle_input_split[18]: {puzzle_input_split[18]}")
print(f"puzzle_input_split[19]: {puzzle_input_split[19]}")
print(f"puzzle_input_split[20]: {puzzle_input_split[20]}")

puzzle_input_split[18]: 1721726623 2213740950 418202427
puzzle_input_split[19]: 
puzzle_input_split[20]: soil-to-fertilizer map:


In [203]:
list_of_number_line_ranges = [range(test_map_index[k] + 1, test_map_index[k+1] - 1) if k < len(test_map_index) - 1 else range(test_map_index[k] + 1, len(puzzle_input_split)) for k in range(len(test_map_index))]
list_of_number_line_ranges

[range(3, 19),
 range(21, 39),
 range(41, 81),
 range(83, 99),
 range(101, 141),
 range(143, 181),
 range(183, 219)]

In [204]:
list(range(3, 20))

[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [205]:
puzzle_input_split[18]

'1721726623 2213740950 418202427'

In [206]:
puzzle_input_split[19]

''

In [207]:
puzzle_input_split[20]

'soil-to-fertilizer map:'

In [208]:
for k in list_of_number_line_ranges[0]:
    print(puzzle_input_split[k])

1270068015 1235603193 242614277
13415696 1478217470 21049126
825250550 1160341941 75261252
3786189027 1971702238 242038712
3191605340 3433644052 172752250
2389904665 3088515862 345128190
0 1499266596 13415696
1197451933 0 72616082
2139929050 1721726623 249975615
900511802 222541761 147014452
1047526254 72616082 149925679
34464822 980591812 179750129
2735032855 2631943377 456572485
3364357590 3606396302 421831437
214214951 369556213 611035599
1721726623 2213740950 418202427


In [209]:
list(range(1,5))

[1, 2, 3, 4]

In [210]:
for number_line in list_of_number_line_ranges:
    for number in number_line:
        print(puzzle_input_split[number])
    print("")

1270068015 1235603193 242614277
13415696 1478217470 21049126
825250550 1160341941 75261252
3786189027 1971702238 242038712
3191605340 3433644052 172752250
2389904665 3088515862 345128190
0 1499266596 13415696
1197451933 0 72616082
2139929050 1721726623 249975615
900511802 222541761 147014452
1047526254 72616082 149925679
34464822 980591812 179750129
2735032855 2631943377 456572485
3364357590 3606396302 421831437
214214951 369556213 611035599
1721726623 2213740950 418202427

226793587 358613369 356867344
0 1838890301 226793587
2741010192 0 358613369
2257843811 715480713 173982825
3099623561 1264222741 3082010
1810570233 2912833547 326150077
4038242924 3815312886 256724372
2431826636 3268919687 279247493
866869902 1671637223 167253078
3102705571 889463538 374759203
1615333950 2646894858 125679350
2136720310 1550513722 121123501
3477464774 2772574208 140259339
1034122980 2065683888 581210970
2711074129 3238983624 29936063
583660931 1267304751 283208971
1741013300 3548167180 69556933
38153

In [211]:
test_matrix_of_mapping_indicators_str = [[puzzle_input_split[number_line] for number_line in number_line_range] for number_line_range in list_of_number_line_ranges]
test_matrix_of_mapping_indicators_str

[['1270068015 1235603193 242614277',
  '13415696 1478217470 21049126',
  '825250550 1160341941 75261252',
  '3786189027 1971702238 242038712',
  '3191605340 3433644052 172752250',
  '2389904665 3088515862 345128190',
  '0 1499266596 13415696',
  '1197451933 0 72616082',
  '2139929050 1721726623 249975615',
  '900511802 222541761 147014452',
  '1047526254 72616082 149925679',
  '34464822 980591812 179750129',
  '2735032855 2631943377 456572485',
  '3364357590 3606396302 421831437',
  '214214951 369556213 611035599',
  '1721726623 2213740950 418202427'],
 ['226793587 358613369 356867344',
  '0 1838890301 226793587',
  '2741010192 0 358613369',
  '2257843811 715480713 173982825',
  '3099623561 1264222741 3082010',
  '1810570233 2912833547 326150077',
  '4038242924 3815312886 256724372',
  '2431826636 3268919687 279247493',
  '866869902 1671637223 167253078',
  '3102705571 889463538 374759203',
  '1615333950 2646894858 125679350',
  '2136720310 1550513722 121123501',
  '3477464774 27725742

In [212]:
test_matrix_of_mapping_indicators_str[0]

['1270068015 1235603193 242614277',
 '13415696 1478217470 21049126',
 '825250550 1160341941 75261252',
 '3786189027 1971702238 242038712',
 '3191605340 3433644052 172752250',
 '2389904665 3088515862 345128190',
 '0 1499266596 13415696',
 '1197451933 0 72616082',
 '2139929050 1721726623 249975615',
 '900511802 222541761 147014452',
 '1047526254 72616082 149925679',
 '34464822 980591812 179750129',
 '2735032855 2631943377 456572485',
 '3364357590 3606396302 421831437',
 '214214951 369556213 611035599',
 '1721726623 2213740950 418202427']

In [213]:
list(map(lambda x: list(map(int, x.split(" "))), test_matrix_of_mapping_indicators_str[0]))

[[1270068015, 1235603193, 242614277],
 [13415696, 1478217470, 21049126],
 [825250550, 1160341941, 75261252],
 [3786189027, 1971702238, 242038712],
 [3191605340, 3433644052, 172752250],
 [2389904665, 3088515862, 345128190],
 [0, 1499266596, 13415696],
 [1197451933, 0, 72616082],
 [2139929050, 1721726623, 249975615],
 [900511802, 222541761, 147014452],
 [1047526254, 72616082, 149925679],
 [34464822, 980591812, 179750129],
 [2735032855, 2631943377, 456572485],
 [3364357590, 3606396302, 421831437],
 [214214951, 369556213, 611035599],
 [1721726623, 2213740950, 418202427]]

In [214]:
def get_mapping_lines_old(puzzle_input_split: list[str]) -> list[int]:
    len_puzzle_input_split = len(puzzle_input_split)
    map_texts, map_lines = zip(*[(x, i) for i, x in enumerate(puzzle_input_split) if "map" in x])
    list_of_number_line_ranges = [range(map_lines[k] + 1, map_lines[k+1] - 1) if k < len(map_lines) - 1 else range(map_lines[k] + 1, len(puzzle_input_split)) for k in range(len(map_lines))]
    
    matrix_of_mapping_indicators_str = [[puzzle_input_split[number_line] for number_line in number_line_range] for number_line_range in list_of_number_line_ranges]

    # list(map(lambda x: list(map(int, x.split(" "))), matrix_of_mapping_indicators_str[0]))

    # map_texts = [x for x in puzzle_input_split if "map" in x]
    # map_lines = [i for i, x in enumerate(puzzle_input_split) if "map" in x]
    
    mapping_lines_dict = {map_texts[i]: list(map(lambda x: list(map(int, x.split(" "))), matrix_of_mapping_indicators_str[i])) for i in range(len(map_texts))}

    return mapping_lines_dict

In [215]:
def get_mappings(puzzle_input_split: list[str]) -> list[int]:
    len_puzzle_input_split = len(puzzle_input_split)
    map_texts, map_lines = zip(*[(x, i) for i, x in enumerate(puzzle_input_split) if "map" in x])
    list_of_number_line_ranges = [range(map_lines[k] + 1, map_lines[k+1] - 1) if k < len(map_lines) - 1 else range(map_lines[k] + 1, len(puzzle_input_split)) for k in range(len(map_lines))]
    
    matrix_of_mapping_indicators_str = [[puzzle_input_split[number_line] for number_line in number_line_range] for number_line_range in list_of_number_line_ranges]

    # list(map(lambda x: list(map(int, x.split(" "))), matrix_of_mapping_indicators_str[0]))

    # map_texts = [x for x in puzzle_input_split if "map" in x]
    # map_lines = [i for i, x in enumerate(puzzle_input_split) if "map" in x]
    
    mapping_lines_dict = {map_texts[i]: np.array(list(map(lambda x: list(map(int, x.split(" "))), matrix_of_mapping_indicators_str[i]))) for i in range(len(map_texts))}

    return mapping_lines_dict

In [216]:
test_mapping_lines = get_mappings(puzzle_input_split)
test_mapping_lines

{'seed-to-soil map:': array([[1270068015, 1235603193,  242614277],
        [  13415696, 1478217470,   21049126],
        [ 825250550, 1160341941,   75261252],
        [3786189027, 1971702238,  242038712],
        [3191605340, 3433644052,  172752250],
        [2389904665, 3088515862,  345128190],
        [         0, 1499266596,   13415696],
        [1197451933,          0,   72616082],
        [2139929050, 1721726623,  249975615],
        [ 900511802,  222541761,  147014452],
        [1047526254,   72616082,  149925679],
        [  34464822,  980591812,  179750129],
        [2735032855, 2631943377,  456572485],
        [3364357590, 3606396302,  421831437],
        [ 214214951,  369556213,  611035599],
        [1721726623, 2213740950,  418202427]]),
 'soil-to-fertilizer map:': array([[ 226793587,  358613369,  356867344],
        [         0, 1838890301,  226793587],
        [2741010192,          0,  358613369],
        [2257843811,  715480713,  173982825],
        [3099623561, 126422274

In [217]:
test_mapping_texts = [x for x in puzzle_input_split if "map" in x]
test_mapping_texts

['seed-to-soil map:',
 'soil-to-fertilizer map:',
 'fertilizer-to-water map:',
 'water-to-light map:',
 'light-to-temperature map:',
 'temperature-to-humidity map:',
 'humidity-to-location map:']

In [218]:
def get_mapping_texts(puzzle_input_split: list[str]) -> list[str]:
    mapping_texts = np.array([x for x in puzzle_input_split if "map" in x], dtype = np.str_)
    return mapping_texts

Let's get the chain of mapping

In [219]:
test_mapping_texts

['seed-to-soil map:',
 'soil-to-fertilizer map:',
 'fertilizer-to-water map:',
 'water-to-light map:',
 'light-to-temperature map:',
 'temperature-to-humidity map:',
 'humidity-to-location map:']

In [220]:
test_mapping_texts[-1]

'humidity-to-location map:'

In [221]:
[x for x in test_mapping_texts if 'location' in x]

['humidity-to-location map:']

In [222]:
reverse_mapping_order_texts = ['location']
reverse_mapping_order_indices = [i for i, x in enumerate(test_mapping_texts) if "location" in x]

In [223]:
reverse_mapping_order_indices

[6]

In [224]:
[x for x in test_mapping_texts if reverse_mapping_order_texts[-1] in x]

['humidity-to-location map:']

In [225]:
# test_mapping_texts_example = test_mapping_texts[-1]
test_mapping_texts_example = [x for x in test_mapping_texts if reverse_mapping_order_texts[-1] in x][0]

In [226]:
test_mapping_texts_example

'humidity-to-location map:'

In [227]:
test_mapping_texts_example_space_split = test_mapping_texts_example.split(" ")
test_mapping_texts_example_space_split

['humidity-to-location', 'map:']

In [228]:
test_mapping_texts_example_space_split_isolated = test_mapping_texts_example_space_split[0]
test_mapping_texts_example_space_split_isolated

'humidity-to-location'

In [229]:
test_mapping_texts_example_source_destination_tag = tuple(test_mapping_texts_example_space_split_isolated.split("-to-"))
test_mapping_texts_example_source_destination_tag

('humidity', 'location')

In [230]:
test_mapping_texts_example_source_destination_tag[0]

'humidity'

In [231]:
test_mapping_texts_example_get_source_entries = [x for x in test_mapping_texts if test_mapping_texts_example_source_destination_tag[0] in x]
test_mapping_texts_example_get_source_entries

['temperature-to-humidity map:', 'humidity-to-location map:']

In [232]:
[(i, x) for i, x in enumerate(test_mapping_texts) if test_mapping_texts_example_source_destination_tag[0] in x and x != test_mapping_texts_example]

[(5, 'temperature-to-humidity map:')]

In [233]:
[] + ["location"]

['location']

In [234]:
def split_mapping_text(mapping_text: str) -> tuple[str]:
    source, destination = tuple(mapping_text.split(" ")[0].split("-to-"))
    return source, destination

In [235]:
test_mapping_texts[0]

'seed-to-soil map:'

In [236]:
split_mapping_text(test_mapping_texts[0])

('seed', 'soil')

In [237]:
def get_index_split_mapping(mapping_texts: list[str]) -> dict:
    return {i: split_mapping_text(x) for i, x in enumerate(mapping_texts)}

In [238]:
get_index_split_mapping(test_mapping_texts)

{0: ('seed', 'soil'),
 1: ('soil', 'fertilizer'),
 2: ('fertilizer', 'water'),
 3: ('water', 'light'),
 4: ('light', 'temperature'),
 5: ('temperature', 'humidity'),
 6: ('humidity', 'location')}

In [239]:
def get_destination_source_index_dict(mapping_texts: list[str]) -> dict:
    # return {destination: (source, index) for index, (source, destination) in enumerate(mapping_texts)}
    destination_source_pairs = [split_mapping_text(x) for x in mapping_texts]
    return {destination: (source, index) for index, (source, destination) in enumerate(destination_source_pairs)}

In [240]:
get_destination_source_index_dict(test_mapping_texts)

{'soil': ('seed', 0),
 'fertilizer': ('soil', 1),
 'water': ('fertilizer', 2),
 'light': ('water', 3),
 'temperature': ('light', 4),
 'humidity': ('temperature', 5),
 'location': ('humidity', 6)}

In [241]:
def get_mapping_path_old(mapping_texts: list[str]):

    # Convert mapping_texts to np array if it is not already
    if isinstance(mapping_texts, list) and all(isinstance(item, str) for item in mapping_texts):
        mapping_texts_np = np.array(mapping_texts, dtype = np.str_)
    elif isinstance(mapping_texts, np.ndarray) and mapping_texts.dtype.type is np.str_:
        mapping_texts_np = mapping_texts
    else:
        raise ValueError("Argument must be a list of strings or a NumPy array of strings")

    while_counter = 0
    while_max_iter = 1000

    destination_source_index_dict = get_destination_source_index_dict(mapping_texts_np)
    reverse_destination_path_text = []
    reverse_mapping_path_index = []

    reverse_destination_path_text += ["location"]

    while reverse_destination_path_text[-1] != "seed" and while_counter < while_max_iter:
        reverse_mapping_path_index += [destination_source_index_dict[reverse_destination_path_text[-1]][1]]
        reverse_destination_path_text += [destination_source_index_dict[reverse_destination_path_text[-1]][0]]
        while_counter += 1
    
    if while_counter == while_max_iter:
        raise RuntimeError("While loop did not find 'seed' before reaching the maximally allowed iterations.")
    
    mapping_path_index = list(reversed(reverse_mapping_path_index))
    mapping_path_text = mapping_texts_np[mapping_path_index] # using mapping_texts_np is numpy array for indexing

    return mapping_path_text, mapping_path_index


In [242]:
def get_mapping_path(puzzle_input_split: list[str]):

    mapping_texts_np = get_mapping_texts(puzzle_input_split)

    while_counter = 0
    while_max_iter = 1000

    destination_source_index_dict = get_destination_source_index_dict(mapping_texts_np)
    reverse_destination_path_text = []
    reverse_mapping_path_index = []

    reverse_destination_path_text += ["location"]

    while reverse_destination_path_text[-1] != "seed" and while_counter < while_max_iter:
        reverse_mapping_path_index += [destination_source_index_dict[reverse_destination_path_text[-1]][1]]
        reverse_destination_path_text += [destination_source_index_dict[reverse_destination_path_text[-1]][0]]
        while_counter += 1
    
    if while_counter == while_max_iter:
        raise RuntimeError("While loop did not find 'seed' before reaching the maximally allowed iterations.")
    
    mapping_path_index = list(reversed(reverse_mapping_path_index))
    mapping_path_text = mapping_texts_np[mapping_path_index] # using mapping_texts_np is numpy array for indexing

    return mapping_path_text, mapping_path_index

In [243]:
test_mapping_texts_np = np.array(test_mapping_texts)
test_mapping_texts_np

array(['seed-to-soil map:', 'soil-to-fertilizer map:',
       'fertilizer-to-water map:', 'water-to-light map:',
       'light-to-temperature map:', 'temperature-to-humidity map:',
       'humidity-to-location map:'], dtype='<U28')

In [244]:
test_mapping_texts_np[[1,2,3]]

array(['soil-to-fertilizer map:', 'fertilizer-to-water map:',
       'water-to-light map:'], dtype='<U28')

In [245]:
get_mapping_path_old(test_mapping_texts_np)

(array(['seed-to-soil map:', 'soil-to-fertilizer map:',
        'fertilizer-to-water map:', 'water-to-light map:',
        'light-to-temperature map:', 'temperature-to-humidity map:',
        'humidity-to-location map:'], dtype='<U28'),
 [0, 1, 2, 3, 4, 5, 6])

### doing the mapping

In [246]:
test_seed = 2041142901

In [247]:
test_mapping_example = test_mapping_lines["seed-to-soil map:"]
test_mapping_example

array([[1270068015, 1235603193,  242614277],
       [  13415696, 1478217470,   21049126],
       [ 825250550, 1160341941,   75261252],
       [3786189027, 1971702238,  242038712],
       [3191605340, 3433644052,  172752250],
       [2389904665, 3088515862,  345128190],
       [         0, 1499266596,   13415696],
       [1197451933,          0,   72616082],
       [2139929050, 1721726623,  249975615],
       [ 900511802,  222541761,  147014452],
       [1047526254,   72616082,  149925679],
       [  34464822,  980591812,  179750129],
       [2735032855, 2631943377,  456572485],
       [3364357590, 3606396302,  421831437],
       [ 214214951,  369556213,  611035599],
       [1721726623, 2213740950,  418202427]])

In [248]:
test_mapping_example_np = np.array(test_mapping_example)

In [249]:
test_mapping_example_np[:,1]

array([1235603193, 1478217470, 1160341941, 1971702238, 3433644052,
       3088515862, 1499266596,          0, 1721726623,  222541761,
         72616082,  980591812, 2631943377, 3606396302,  369556213,
       2213740950])

In [250]:
test_destination_ranges, test_source_ranges = get_destination_source_ranges(source_range_start_list=test_mapping_example_np[:,0], destination_range_start_list=test_mapping_example_np[:,1], range_length_list=test_mapping_example_np[:,2])

In [251]:
get_destination_number(source_number = test_seed, destination_ranges=test_destination_ranges, source_ranges = test_source_ranges)

2533157228

In [252]:
test_dict = {'a':1, 'b': 2, 'c': 3}

In [253]:
test_dict['a']

1

In [146]:
def add_1(x: int) -> int:
    return x + 1

In [149]:
test_number_list = [1,2,3,4]

In [151]:
test_number_list

[6, 7, 8, 9]

In [150]:
for i in range(5):
    test_number_list = [add_1(x) for x in test_number_list]

In [254]:
def solve_puzzle(puzzle_input_split):

    seeds = get_seeds(puzzle_input_split)
    mappings = get_mappings(puzzle_input_split)
    mapping_path_text, _ = get_mapping_path(puzzle_input_split)
    source_numbers = seeds.copy()
    
    for map_text in mapping_path_text:
        print(map_text)
        current_map = mappings[map_text]
        current_destination_ranges, current_source_ranges = get_destination_source_ranges(source_range_start_list=current_map[:,0], destination_range_start_list=current_map[:,1], range_length_list=current_map[:,2])
        
        source_numbers = [get_destination_number(source_number = source_number, destination_ranges=current_destination_ranges, source_ranges = current_source_ranges) for source_number in source_numbers]
    
    min_mapping_value = min(source_numbers)
    min_mapping_value_index = source_numbers.index(min_mapping_value)

    return seeds[min_mapping_value_index]

In [ ]:
# !!! 2025_08_04: ANSWER IS WRONG. 2041142901 IS TOO HIGH
solve_puzzle(puzzle_input_split)

seed-to-soil map:
soil-to-fertilizer map:
fertilizer-to-water map:
water-to-light map:
light-to-temperature map:
temperature-to-humidity map:
humidity-to-location map:


2041142901